<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/whisper-large-v3-turbo/whisper_large_v3_turbo-cv16.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install jiwer datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
ds = load_dataset("mozilla-foundation/common_voice_16_1", "ko", split="test")
samples = ds['audio']


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_1.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
pipeline_output = pipe(samples, generate_kwargs={"task": "transcribe", "language": "ko"})

In [ ]:

# pipeline_output
results = [ f['text'] for f in pipeline_output ]
print(results)

In [ ]:
import re
from jiwer import wer, cer

hyp = results
ref = ds['sentence']

def preprocess_text_list(sentence_list):
  # 각 문장에서 알파벳, 숫자, 띄어쓰기를 제외한 모든 문자 제거
  return [re.sub(r'[^가-힣a-zA-Z0-9\s]', '', sentence) for sentence in sentence_list]



# 전처리된 텍스트로 WER, CER 계산
ground_truth_processed = preprocess_text_list(ref)
hypothesis_processed = preprocess_text_list(hyp)

wer_value = wer(ground_truth_processed, hypothesis_processed)
cer_value = cer(ground_truth_processed, hypothesis_processed)

In [ ]:
print(ground_truth_processed)
print(hypothesis_processed)

In [ ]:
print(wer_value, cer_value)

In [ ]:
for h, r in zip(hypothesis_processed, ground_truth_processed):
  print(f'{h} | {r}')

In [ ]:
# 실행 시간 측정을 위한 래퍼 함수 생성
import timeit

def sample_function():
    pipeline_output = pipe(samples, generate_kwargs={"task": "transcribe", "language": "ko"})

execution_time = timeit.timeit(sample_function, number=1)  # number=100은 실행 횟수

# 결과 출력
print("Execution time:", execution_time, "seconds")

In [ ]:
from datasets import Audio
import jiwer

def prepare_dataset(batch):
    audio = batch['audio']
    batch["length"] = float(audio["array"].size / audio["sampling_rate"])
    #batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    #batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

ds_test = ds.map(prepare_dataset)

In [ ]:
len = 0.0
for l in ds_test['length']:
    len += float(l)

rtfx = float(execution_time / len)

print(rtfx)